In [1]:
#Install The required libraries and import them.

import pandas as pd
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType,StructField, StringType, IntegerType
from pyspark.sql.functions import * 

In [2]:
# Initialize Spark session
spark = SparkSession.builder \
    .appName("CSV to DB Migration") \
    .getOrCreate()